In [14]:
import tabulate
import yaml
import pandas as pd
import ast
from util import *
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import warnings
warnings.filterwarnings("ignore")

In [15]:
data = pd.read_csv("data_with_ff.csv")

In [16]:
sys_cl_and_ff = data.form_factor.unique()

In [17]:
sys_cl_and_ff

array(["{'diffuse': {'guinier_porod'}}", "{'diffuse': {'spherical'}}",
       "{'diffuse': {'spherical', 'guinier_porod'}}",
       "{'diffuse': {'spherical', 'guinier_porod'}, 'disordered': {'spherical'}}",
       "{'unidentified': set()}",
       "{'diffuse': {'spherical'}, 'disordered': {'spherical'}}",
       "{'diffuse': {'guinier_porod'}, 'disordered': {'guinier_porod'}}"],
      dtype=object)

In [18]:
sys_cl_and_ff = [a for a in sys_cl_and_ff if "unidentified" not in a]

In [19]:
table = []
table.append([None, None, 'Cls M', None, None, None, None,"Regr M", None])
table.append(['Sys Cl', "Form", 'Pr', 'Rec', 'Accur', 'F1',"Regr Par", 'MAE', 'R^2', '# samp'])

In [20]:
print(tabulate.tabulate(table,tablefmt="fancy_grid"))

╒════════╤══════╤═══════╤═════╤═══════╤════╤══════════╤════════╤═════╤════════╕
│        │      │ Cls M │     │       │    │          │ Regr M │     │        │
├────────┼──────┼───────┼─────┼───────┼────┼──────────┼────────┼─────┼────────┤
│ Sys Cl │ Form │ Pr    │ Rec │ Accur │ F1 │ Regr Par │ MAE    │ R^2 │ # samp │
╘════════╧══════╧═══════╧═════╧═══════╧════╧══════════╧════════╧═════╧════════╛


In [21]:
yml_path = '../xrsdkit_modeling/flowreactor_pd_nanoparticles/models/training_summary.yml'
with open(yml_path, 'r') as stream:
        summary_loaded = yaml.load(stream)

In [22]:
regrs = summary_loaded['REGRESSORS']

In [23]:
for cl_ff in sys_cl_and_ff:
    related_data = data[(data['form_factor'] == cl_ff) | (data['form_factor_pr'] == cl_ff)]
    y_true = related_data.form_factor.to_list()
    y_pr = related_data.form_factor_pr.to_list()
    cl_ff_tr = [1 if a == cl_ff else 0 for a in y_true ]
    n_samples = sum(cl_ff_tr)
    cl_ff_pr = [1 if a == cl_ff else 0 for a in y_pr ]
    pr = round(precision_score(cl_ff_tr, cl_ff_pr),2)
    rec = round(recall_score(cl_ff_tr, cl_ff_pr),2)
    acc = round(accuracy_score(cl_ff_tr, cl_ff_pr),2)
    f1 = round(f1_score(cl_ff_tr, cl_ff_pr),2)
    
    cl_ff_d = ast.literal_eval(cl_ff)
    i_p = 0
    s_cl = ''
    s_cl_show = ''
    forms = []
    for k,v in cl_ff_d.items():
        n = len(v)
        for i in range(n):
            s_cl_show += 'Pop {} : '.format(i_p)
            s_cl+=k
            s_cl_show +=k
            s_cl+="__"
            s_cl_show+="\n"
            i_p+=1
        for f in v:
            forms.append(f)
        
    s_cl= s_cl[ : -2]
    
    forms_str = "\n".join(forms)
    
    reg_params = []
    maes = []
    r2s = []
    
    strs = s_cl.split("__")
    rm = regrs[s_cl]
    for i in range(len(forms)):
        st = strs[i]
        form = forms[i]
        regrs[s_cl]
        # find the pop with coresponding structure and form:
        for k,v in rm.items():
            if form == 'guinier_porod':
                if 'pop' in k and form in v.keys():
                    pop_number = int(k[-1])
                    if st == strs[pop_number]:
                        reg_params.append('rg')
                        try:
                            maes.append(str(round(v[form]['rg']['scores']['MAE'],2)))
                            r2s.append(str(round(v[form]['rg']['scores']['coef_of_determination'],2)))
                        except:
                            maes.append("-")
                            r2s.append("-")
            elif form == 'spherical':
                if 'pop' in k and form in v.keys():
                    pop_number = int(k[-1])
                    if st == strs[pop_number]:
                        reg_params.append('r')
                        reg_params.append('sigma')
                        try:
                            maes.append(str(round(v[form]['r']['scores']['MAE'],2)))
                            r2s.append(str(round(v[form]['r']['scores']['coef_of_determination'],2)))
                            maes.append(str(round(v[form]['distribution']['r_normal']['sigma']['scores']['MAE'],2)))
                            r2s.append(str(round(v[form]['distribution']['r_normal']['sigma']['scores']['coef_of_determination'],2)))
                        except:
                            maes.append("-")
                            r2s.append("-")
                            maes.append("-")
                            r2s.append("-")
            #else:
                # add support for crystalline
    reg_params = "\n".join(reg_params)
    maes = "\n".join(maes)
    r2s = "\n".join(r2s)
    table.append([s_cl_show,forms_str, pr, rec, acc, f1, reg_params, maes, r2s, n_samples])

In [24]:
print(tabulate.tabulate(table,tablefmt="fancy_grid"))

╒════════════════════╤═══════════════╤═══════╤══════╤═══════╤══════╤══════════╤════════╤══════╤════════╕
│                    │               │ Cls M │      │       │      │          │ Regr M │      │        │
├────────────────────┼───────────────┼───────┼──────┼───────┼──────┼──────────┼────────┼──────┼────────┤
│ Sys Cl             │ Form          │ Pr    │ Rec  │ Accur │ F1   │ Regr Par │ MAE    │ R^2  │ # samp │
├────────────────────┼───────────────┼───────┼──────┼───────┼──────┼──────────┼────────┼──────┼────────┤
│ Pop 0 : diffuse    │ guinier_porod │ 0.87  │ 0.71 │ 0.65  │ 0.78 │ rg       │ 0.23   │ 0.9  │ 28     │
├────────────────────┼───────────────┼───────┼──────┼───────┼──────┼──────────┼────────┼──────┼────────┤
│ Pop 0 : diffuse    │ spherical     │ 1.0   │ 0.11 │ 0.11  │ 0.2  │ r        │ 0.24   │ 0.88 │ 45     │
│                    │               │       │      │       │      │ sigma    │ 0.56   │ 0.41 │        │
├────────────────────┼───────────────┼───────┼──────┼──

In [12]:
with open('table.txt', 'w') as outputfile:
    outputfile.write(tabulate.tabulate(table,tablefmt="fancy_grid"))

In [13]:
tabulate.LATEX_ESCAPE_RULES={}
with open('table_latex.tex', 'w') as outputfile:
    outputfile.write(tabulate.tabulate(table,tablefmt="latex_raw"))